# Task Description

You will be required to produce code to process and transform some sample data.
The sample data is in the file called `data.json`.
There are also some duplicate rows.

The produced code should be able to acheive the following

1. remove duplicates over the columns `id` and `created_at` (considered simultaneously)
2. compute the rank of each user's `user_score` within each age group and output the rank in a new column called `sub_group_rank`
3. process the column `widget_list` by
    1. flattening the list items i.e. each item in the list is put into its own row
    2. extracting the values in the JSON elements into their own columns called `widget_name` and `widget_amount`
4. anonymize the column `email` and output the anonymized version in a new column `email_anon`.
This column `email_anon` should have the following properties.
    1. given an anonymized value the original value can be recovered
5. create a new table that is an inverted index that gives, for each country in `location,` which `id`s are located in that country
6. write the processed tables/data into separate `parquet` file(s).
Exactly how the files/tables are organized is not as important as having all the data present.

Your code will be evaluated for correctness, scalability and maintainability.

In [4]:
!pip install wheel pandas pandasql

  Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)


In [5]:
import pandas as pd
import pandasql as pds

In [8]:
data = pd.read_json('data.json', lines=True)

In [9]:
data.head()

,id,email,age_group,user_score,revenue,widget_list,location,created_at
0,25abf8a8-9d11-4e55-a003-246e458e9fb3,fguinness0@yolasite.com,2,0.467348,351.08,"[{'name': 'Eumetopias jubatus', 'amount': 7052...",Indonesia,2020-01-31 14:50:26+00:00
1,6bf052c9-1307-4b26-a24d-9c47bf747c87,fdebenham1@si.edu,1,0.929755,801.00,"[{'name': 'Bassariscus astutus', 'amount': 441...",Greece,2020-07-16 18:32:48+00:00
2,c3dc4870-463e-49d3-aba7-383cace03cd5,lleband2@hao123.com,3,0.473031,637.77,[],China,2019-09-26 05:26:24+00:00
3,42c18d61-34bb-4b27-a2cf-f43728c690f7,sbartolommeo3@imgur.com,4,0.315440,588.28,"[{'name': 'Manouria emys', 'amount': 72401}, {...",Finland,2020-01-17 09:26:34+00:00
4,e4c7b0ea-466e-4ada-afef-2f1d475be165,ebraben4@xing.com,3,0.009215,191.75,"[{'name': 'Mazama americana', 'amount': 35949}...",Armenia,2020-01-29 21:55:14+00:00


In [24]:
groupby = data.groupby(['id', 'created_at']).size()

In [22]:
groupby.filter(lambda x: len(x) > 1) 

TypeError: 'function' object is not iterable